In [3]:
import re
import requests
import pandas as pd
import language_tool_python
from hanspell import spell_checker
from soynlp.normalizer import *

from nltk.tokenize import word_tokenize

In [60]:
#pip install soynlp

     |████████████████████████████████| 416 kB 9.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
ETRI_API_KEY = "c7d165fa-4ecf-4047-967d-b593a628c568"

In [7]:
# 불용어 리스트
stopwords = ["이", "그", "저", "의", "을", "를", "은", "는", "에", "가", "와", "으로", "에서","추천"]


In [8]:
#ETRI API를 사용하여 맞춤법 교정하기
def correct_text_etri(text):
    url = "http://aiopen.etri.re.kr:8000/WiseNLU"
    headers = {
        "Authorization": ETRI_API_KEY,
        "Content-Type": "application/json"
    }
    data = {
        "argument": {
            "text": text,
            "analysis_code": "morp"  # 형태소 분석 코드, 맞춤법 및 문법 교정 포함
        }
    }

    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        result = response.json()
        if "return_object" in result and "sentence" in result["return_object"]:
            sentences = result["return_object"]["sentence"]
            corrected_text = " ".join([sent["text"] for sent in sentences])
            return corrected_text
        else:
            return "결과가 없습니다."
    else:
        return f"Error: {response.status_code}"


In [9]:
def correct_text_etri2(text):
    url = "http://aiopen.etri.re.kr:8000/WiseNLU"
    headers = {
        "Authorization": ETRI_API_KEY,
        "Content-Type": "application/json"
    }
    data = {
        "argument": {
            "text": text,
            "analysis_code": "morp"
        }
    }

    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        result = response.json()
        words = []
        for sentence in result["return_object"]["sentence"]:
            for morp in sentence["morp"]:
                words.append(morp["lemma"])
        return " ".join(words)
    else:
        return f"Error: {response.status_code}"

In [10]:
# ETRI API를 사용하여 유의어 가져오기
def get_synonyms(word):
    url = "http://aiopen.etri.re.kr:8000/WiseWWN/Word"
    headers = {"Authorization": ETRI_API_KEY, "Content-Type": "application/json"}
    data = {"request_id": "reserved field", "argument": {"word": word}}
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        result = response.json()
        return result.get("return_object", {}).get("WWN WordInfo", {}).get("synonym", [])
    else:
        print(f"Error: {response.status_code}")
        return []

In [11]:
def get_synonyms_from_etri(word):
    url = "http://aiopen.etri.re.kr:8000/WiseNLU"
    headers = {"Authorization": ETRI_API_KEY, "Content-Type": "application/json"}
    data = {
        "argument": {
            "text": word,
            "analysis_code": "morp"
        }
    }

    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        result = response.json()
        return [morp['lemma'] for sentence in result['return_object']['sentence']
                for morp in sentence['morp'] if morp['type'] == 'NNG']  # 명사만 추출
    else:
        return []

print(get_synonyms_from_etri("활동"))

['활동']


In [12]:
def get_synonyms2(word):
    url = "http://aiopen.etri.re.kr:8000/WiseNLU"
    headers = {
        "Authorization": ETRI_API_KEY,
        "Content-Type": "application/json"
    }
    data = {
        "argument": {
            "text": word,
            "analysis_code": "morp"
        }
    }

    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        result = response.json()
        # API 응답 데이터 구조에 따라 유의어 추출
        word_info = result.get("return_object", {}).get("sentence", [])
        synonyms = []
        for sentence in word_info:
            for morp in sentence.get("morp", []):
                if morp.get("type") == "NNG":  # 명사(NNG) 추출 예시
                    synonyms.append(morp.get("lemma"))
        return synonyms
    else:
        print(f"Error: {response.status_code}")
        return []

In [21]:
# 전처리 함수
def preprocess_text(text):
    # 1. 맞춤법 교정
    text = spell_checker.check(text).checked


    # 2. 특수문자 및 숫자 제거
    text = re.sub(r"[^가-힣0-9\s]", "", text)
    
    #반복되는 말 정규화
    text = repeat_normalize(text, num_repeats=2)

    # 3. 불용어 제거
    words = text.split()
    words = [word for word in words if word not in stopwords]

    # 4. 유의어 교체
    synonym_map = {}
    for word in words:
        synonyms = get_synonyms2(word)
        if synonyms:
            synonym_map[word] = synonyms[0]  # 첫 번째 유의어로 교체

    replaced_words = [synonym_map.get(word, word) for word in words]
    return " ".join(replaced_words)

 # 유의어 교체 없이 반환
   # return " ".join(words)

In [15]:
result = spell_checker.check('안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.')
result.as_dict()  # dict로 출력

{'result': True,
 'original': '안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.',
 'checked': '안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다.',
 'errors': 4,
 'words': OrderedDict([('안녕하세요.', 2),
              ('저는', 0),
              ('한국인입니다.', 2),
              ('이', 2),
              ('문장은', 2),
              ('한글로', 0),
              ('작성됐습니다.', 1)]),
 'time': 0.1473238468170166}

In [17]:
# 데이터 입력
book_data = [
    {"제목": "소년이 온다","작가": "한강", "내용": " 소설은 6개의 장으로 나누어져 있었는데요 각 장마다 다른 인물의 시선으로 이야기가 진행되고 있었어요 주인공들은 광주에서 일어난 시위 사건 속에 있던 사람들로 각자의 슬픔을 안고 살아가고 있습니다."},
    {"제목": "책 제목 2", "작가": "작가 이름 2", "내용": "다른 주제를 다룬 책입니다."},
    {"제목": "책 제목 3", "작가": "작가 이름 3", "내용": "흥미로운 이야기가 담긴 책입니다."}
]

# DataFrame 생성
df = pd.DataFrame(book_data)

# CSV 파일로 저장
csv_file = "books.csv"
df.to_csv(csv_file, index=False, encoding="utf-8-sig")

print(f"CSV 파일 생성 완료: {csv_file}")

CSV 파일 생성 완료: books.csv


In [18]:
csv_file = "books.csv"
df = pd.read_csv(csv_file, encoding="utf-8-sig")

# 내용에 전처리 함수 적용
df["전처리된 내용"] = df["내용"].apply(preprocess_text)

# 결과 확인
print(df)

# 전처리된 데이터를 새로운 CSV 파일로 저장
processed_csv_file = "processed_books.csv"
df.to_csv(processed_csv_file, index=False, encoding="utf-8-sig")
print(f"전처리된 CSV 파일 저장 완료: {processed_csv_file}")

       제목       작가                                                 내용  \
0  소년이 온다       한강   소설은 6개의 장으로 나누어져 있었는데요 각 장마다 다른 인물의 시선으로 이야기가...   
1  책 제목 2  작가 이름 2                                    다른 주제를 다룬 책입니다.   
2  책 제목 3  작가 이름 3                                 흥미로운 이야기가 담긴 책입니다.   

                                             전처리된 내용  
0  소설은 6개의 장으로 나누어져 있었는데요 각 장마다 다른 인물의 시선으로 이야기가 ...  
1                                     다른 주제를 다룬 책입니다  
2                                  흥미로운 이야기가 담긴 책입니다  
전처리된 CSV 파일 저장 완료: processed_books.csv


In [22]:
# 예제 입력 텍스트
user_text = "민주화 운동과정에서 희생된 시민들의 이야기를 쓴 소설을 추천해줘ㅋㅋㅋㅋㅋㅋㅋ ㅎㅎ"

# 전처리 수행
preprocessed_text = preprocess_text(user_text)

print(f"전처리 결과: {preprocessed_text}")

전처리 결과: 민주 운동 과정 희생 시민 이야기 쓴 소설 추천
